In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20)

In [ ]:
library(tidyverse)
library(rstatix)
library(ggplot2)
library(hrbrthemes)
library(irr)

In [ ]:
t_fast <- readRDS("fastsurfer_absolute_summed_vols.rds") %>%


gather(key='ROI', value='vol_fast', -IDs)
t_fast %>% head()

In [ ]:
t_free <-readRDS("freesurfer_absolute_summed_vols.rds") %>%
mutate(IDs = gsub('([0-9a-zA-Z_-]*).reconall','\\1', IDs)) %>%
gather(key='ROI', value='vol_free', -IDs)

t_free %>% head()

In [ ]:
res = T
for (i in t_free$ROI %>% unique()) {
    res & (i %in% (t_free$ROI %>% unique()))
    }


In [ ]:
data <- merge(t_free, t_fast, by=c('IDs', 'ROI'))

In [ ]:
qc <- read.table('/home/ibrazug/Dokumente/KindersegV2/QC_SC_RESULT(final).csv', sep=',', header=T) %>%
select(c(subject, qc_inclusion)) %>% rename(IDs = subject) 


In [ ]:
data <- merge(data,
              qc,
              by='IDs') %>% filter(qc_inclusion == 'yes') %>% select(-qc_inclusion)

In [ ]:
data %>%
  group_by(ROI) %>%
  summarize(
    agreement = icc(cbind(vol_fast, vol_free), "twoway", 'agreement')$value,
    lCI = icc(cbind(vol_fast, vol_free), "twoway", 'agreement')$lbound,
    uCI = icc(cbind(vol_fast, vol_free), "twoway", 'agreement')$ubound,
    mean_vol = mean((vol_fast + vol_free) / 2, na.rm = TRUE),
    sd_mean_vol = sd((vol_fast + vol_free) / 2, na.rm = TRUE),
  )

